In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from time import time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_predict,cross_val_score
from sklearn.metrics import roc_curve,auc,precision_score,recall_score,f1_score,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier

raw_train = pd.read_csv('C:/Users/whileduck/Desktop/code/dataset/train.csv')
raw_test = pd.read_csv('C:/Users/whileduck/Desktop/code/dataset/test.csv')

pre_train = pd.read_csv('C:/Users/whileduck/Desktop/code/daycon_train.csv')
pre_test = pd.read_csv('C:/Users/whileduck/Desktop/code/daycon_test.csv')

raw_train = raw_train.drop('ID',axis = 1)
raw_test = raw_test.drop('ID',axis = 1)

raw_x = raw_train.drop('전화해지여부',axis = 1)
raw_y = raw_train['전화해지여부']

pre_x = pre_train.drop('전화해지여부',axis = 1)
pre_y = pre_train['전화해지여부']



In [15]:

x_raw_train,x_raw_test,y_raw_train,y_raw_test = train_test_split(raw_x,raw_y, random_state = 42, test_size = 0.2, stratify = raw_y)
x_pre_train,x_pre_test,y_pre_train,y_pre_test = train_test_split(pre_x,pre_y, random_state = 42, test_size = 0.2, stratify = pre_y)


* 한 번에 알려고 하지 말고 천천히 알아가며, 찾아가보자 

In [17]:
param_lgb = {'max_depth':range(20,31),
             'learning_rate':[0.1,0.5]}

raw_lgbm = LGBMClassifier(n_estimators = 1000)
pre_lgbm = LGBMClassifier()

grid_raw_lgbm = GridSearchCV(estimator = raw_lgbm, param_grid = param_lgb, scoring ='f1_micro', cv = 3, verbose = 2,n_jobs=1)
grid_pre_lgbm = GridSearchCV(estimator = pre_lgbm, param_grid = param_lgb, scoring ='f1_micro', cv = 3, verbose = 2,n_jobs=1)

grid_raw_lgbm.fit(x_raw_train,y_raw_train)
grid_pre_lgbm.fit(x_pre_train,y_pre_train)

Fitting 3 folds for each of 22 candidates, totalling 66 fits
[CV] END ....................learning_rate=0.1, max_depth=20; total time=   1.9s
[CV] END ....................learning_rate=0.1, max_depth=20; total time=   1.8s
[CV] END ....................learning_rate=0.1, max_depth=20; total time=   2.0s
[CV] END ....................learning_rate=0.1, max_depth=21; total time=   2.1s
[CV] END ....................learning_rate=0.1, max_depth=21; total time=   2.1s
[CV] END ....................learning_rate=0.1, max_depth=21; total time=   2.0s
[CV] END ....................learning_rate=0.1, max_depth=22; total time=   2.2s
[CV] END ....................learning_rate=0.1, max_depth=22; total time=   2.2s
[CV] END ....................learning_rate=0.1, max_depth=22; total time=   3.1s
[CV] END ....................learning_rate=0.1, max_depth=23; total time=   2.8s
[CV] END ....................learning_rate=0.1, max_depth=23; total time=   2.6s
[CV] END ....................learning_rate=0.1, 

GridSearchCV(cv=3, estimator=LGBMClassifier(), n_jobs=1,
             param_grid={'learning_rate': [0.1, 0.5],
                         'max_depth': range(20, 31)},
             scoring='f1_micro', verbose=2)

In [18]:
print('원본 데이터의 LGB 파라미터 : ',grid_raw_lgbm.best_params_)
print('전처리 데이터의 LGB 파라미터 :',grid_pre_lgbm.best_params_)

원본 데이터의 LGB 파라미터 :  {'learning_rate': 0.5, 'max_depth': 22}
전처리 데이터의 LGB 파라미터 : {'learning_rate': 0.5, 'max_depth': 20}


In [20]:
raw_lgbm = LGBMClassifier(n_estimators = 1000, learning_rate = 0.5)
pre_lgbm = LGBMClassifier(n_estimators = 1000, learning_rate=  0.5)

param_lgb = {'num_leaves':[100,125,300,500],
             'min_child_samples':[10,20,30]}

grid_raw_lgbm = GridSearchCV(estimator = raw_lgbm, param_grid = param_lgb, scoring ='f1_micro', cv = 3, verbose = 2,n_jobs=1)
grid_pre_lgbm = GridSearchCV(estimator = pre_lgbm, param_grid = param_lgb, scoring ='f1_micro', cv = 3, verbose = 2,n_jobs=1)

grid_raw_lgbm.fit(x_raw_train,y_raw_train)
grid_pre_lgbm.fit(x_pre_train,y_pre_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ...............min_child_samples=10, num_leaves=100; total time=   2.6s
[CV] END ...............min_child_samples=10, num_leaves=100; total time=   2.6s
[CV] END ...............min_child_samples=10, num_leaves=100; total time=   2.6s
[CV] END ...............min_child_samples=10, num_leaves=125; total time=   2.6s
[CV] END ...............min_child_samples=10, num_leaves=125; total time=   2.5s
[CV] END ...............min_child_samples=10, num_leaves=125; total time=   2.7s
[CV] END ...............min_child_samples=10, num_leaves=300; total time=   3.0s
[CV] END ...............min_child_samples=10, num_leaves=300; total time=   2.8s
[CV] END ...............min_child_samples=10, num_leaves=300; total time=   2.9s
[CV] END ...............min_child_samples=10, num_leaves=500; total time=   2.9s
[CV] END ...............min_child_samples=10, num_leaves=500; total time=   3.0s
[CV] END ...............min_child_samples=10, nu

GridSearchCV(cv=3,
             estimator=LGBMClassifier(learning_rate=0.5, n_estimators=1000),
             n_jobs=1,
             param_grid={'min_child_samples': [10, 20, 30],
                         'num_leaves': [100, 125, 300, 500]},
             scoring='f1_micro', verbose=2)

In [21]:
print('원본 데이터의 LGB 파라미터 : ',grid_raw_lgbm.best_params_)
print('전처리 데이터의 LGB 파라미터 :',grid_pre_lgbm.best_params_)

원본 데이터의 LGB 파라미터 :  {'min_child_samples': 10, 'num_leaves': 125}
전처리 데이터의 LGB 파라미터 : {'min_child_samples': 20, 'num_leaves': 100}
